In [1]:
import os
import random
import glob
import cv2

def boxes_intersect(boxA, boxB):
    """
    Check if two boxes intersect.
    Each box is represented as (x1, y1, x2, y2).
    Returns True if the boxes overlap, False otherwise.
    """
    ax1, ay1, ax2, ay2 = boxA
    bx1, by1, bx2, by2 = boxB

    # If one rectangle is on the left side of the other
    if ax2 <= bx1 or bx2 <= ax1:
        return False
    # If one rectangle is above the other
    if ay2 <= by1 or by2 <= ay1:
        return False
    return True

def crop_images_without_blurred_faces(input_dir, output_dir,
                                      crop_size=(512, 512),
                                      seed=42,
                                      blur_threshold=500.0,
                                      max_attempts=50):
    """
    For each image in input_dir, detect blurred faces and then take a random crop
    of size crop_size that does not include any part of any blurred face.
    
    Parameters:
      - input_dir: Path to the directory containing the input images.
      - output_dir: Path to the directory where cropped images will be saved.
      - crop_size: Tuple indicating the (width, height) of the crop.
      - seed: Seed for the random number generator (for reproducibility).
      - blur_threshold: Variance threshold below which a face is considered blurred.
      - max_attempts: Maximum attempts to find a valid crop.
    """
    # Set seed for reproducibility
    random.seed(seed)

    # Create output directory if needed
    os.makedirs(output_dir, exist_ok=True)

    # Gather image files from the input directory (adjust extensions as needed)
    image_extensions = ('*.jpg')
    image_files = []
    for ext in image_extensions:
        image_files.extend(glob.glob(os.path.join(input_dir, ext)))
    image_files = sorted(image_files)

    # Load Haar cascade for face detection
    cascade_path = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
    face_cascade = cv2.CascadeClassifier(cascade_path)
    if face_cascade.empty():
        print("Error loading Haar cascade. Check the path:", cascade_path)
        return

    for image_path in image_files:
        # Read the image
        img = cv2.imread(image_path)
        if img is None:
            print(f"Could not load image {image_path}. Skipping.")
            continue

        height, width = img.shape[:2]
        crop_w, crop_h = crop_size

        # Ensure image is large enough for the desired crop
        if width < crop_w or height < crop_h:
            print(f"Skipping {os.path.basename(image_path)}: image too small ({width}x{height}).")
            continue

        # Convert to grayscale for face detection and blur evaluation
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Detect faces in the full image
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
        print(f"Detected {len(faces)} faces in {os.path.basename(image_path)}.")
        blurred_faces = []

        for (fx, fy, fw, fh) in faces:
            face_roi = gray[fy:fy+fh, fx:fx+fw]
            # Compute the variance of the Laplacian (a measure of sharpness)
            variance = cv2.Laplacian(face_roi, cv2.CV_64F).var()
            if variance < blur_threshold:
                blurred_faces.append((fx, fy, fw, fh))
                # Debug: print(f"Detected blurred face at {(fx,fy,fw,fh)} with variance {variance:.2f}")

        # Attempt to find a valid random crop that does not include any blurred face
        valid_crop_found = False
        for attempt in range(max_attempts):
            crop_x = random.randint(0, width - crop_w)
            crop_y = random.randint(0, height - crop_h)
            crop_box = (crop_x, crop_y, crop_x + crop_w, crop_y + crop_h)

            valid = True  # Assume the crop is valid until proven otherwise

            # Check each blurred face bounding box for any intersection with the crop box.
            for (fx, fy, fw, fh) in blurred_faces:
                face_box = (fx, fy, fx + fw, fy + fh)
                if boxes_intersect(crop_box, face_box):
                    valid = False
                    # Debug: print(f"Crop at {(crop_x, crop_y)} intersects blurred face at {(fx,fy,fw,fh)}")
                    break

            if valid:
                valid_crop_found = True
                break  # Exit the loop if a valid crop is found

        if not valid_crop_found:
            print(f"No valid crop found for {os.path.basename(image_path)} after {max_attempts} attempts. Skipping.")
            continue

        # Crop the image using the valid coordinates
        cropped_img = img[crop_y:crop_y+crop_h, crop_x:crop_x+crop_w]

        # Save the cropped image with the same base filename
        base_name = os.path.basename(image_path)
        output_path = os.path.join(output_dir, base_name)
        cv2.imwrite(output_path, cropped_img)
        print(f"Cropped {base_name} at ({crop_x}, {crop_y}) and saved to {output_path}")

if __name__ == "__main__":
    # Replace with your actual paths. Use raw strings (r"...") or double backslashes on Windows.
    input_folder = r"D:\Download\meta\test for deblurring"    # Folder containing the original images.
    output_folder = r"D:\Download\meta\Cropped and Deblurred"     # Folder to save the cropped images.

    crop_images_without_blurred_faces(input_folder, output_folder)


Could not load image D:\Download\meta\test for deblurring\.. Skipping.
Detected 5 faces in sa_66.jpg.
Cropped sa_66.jpg at (456, 25) and saved to D:\Download\meta\Cropped and Deblurred\sa_66.jpg
Detected 3 faces in sa_7.jpg.
Cropped sa_7.jpg at (759, 563) and saved to D:\Download\meta\Cropped and Deblurred\sa_7.jpg
Detected 3 faces in sa_70.jpg.
Cropped sa_70.jpg at (1003, 228) and saved to D:\Download\meta\Cropped and Deblurred\sa_70.jpg
Detected 5 faces in sa_71.jpg.
Cropped sa_71.jpg at (142, 1508) and saved to D:\Download\meta\Cropped and Deblurred\sa_71.jpg
Detected 8 faces in sa_8.jpg.
Cropped sa_8.jpg at (209, 692) and saved to D:\Download\meta\Cropped and Deblurred\sa_8.jpg
Detected 1 faces in sa_9.jpg.
Cropped sa_9.jpg at (1516, 913) and saved to D:\Download\meta\Cropped and Deblurred\sa_9.jpg


In [9]:
import os
import random
import glob
import cv2

def boxes_intersect(boxA, boxB):
    """
    Check if two boxes intersect.
    Each box is represented as (x1, y1, x2, y2).
    Returns True if the boxes overlap, False otherwise.
    """
    ax1, ay1, ax2, ay2 = boxA
    bx1, by1, bx2, by2 = boxB

    # If one rectangle is completely to the left or above the other, they don't intersect.
    if ax2 <= bx1 or bx2 <= ax1:
        return False
    if ay2 <= by1 or by2 <= ay1:
        return False
    return True

def detect_blurred_regions(gray, block_size = 64, blur_threshold = 100):
    """
    Divide the grayscale image into non-overlapping blocks.
    For each block, compute the variance of the Laplacian.
    Blocks with variance below the blur_threshold are considered blurred.
    
    Returns:
        A list of bounding boxes for blurred blocks, each in the form (x1, y1, x2, y2).
    """
    height, width = gray.shape
    blurred_regions = []
    for y in range(0, height, block_size):
        for x in range(0, width, block_size):
            # Make sure the block does not exceed the image boundaries.
            x_end = min(x + block_size, width)
            y_end = min(y + block_size, height)
            block = gray[y:y_end, x:x_end]
            # Compute the variance of the Laplacian (a measure of sharpness).
            var = cv2.Laplacian(block, cv2.CV_64F).var()
            if var < blur_threshold:
                blurred_regions.append((x, y, x_end, y_end))
    return blurred_regions

def crop_images_avoiding_blur(input_dir, output_dir,
                              crop_size=(512, 512),
                              seed=42,
                              blur_threshold = 100,
                              block_size=64,
                              max_attempts=100):
    """
    For each image in input_dir:
      1. Convert it to grayscale.
      2. Divide it into blocks and detect blurred regions.
      3. Attempt to pick a random crop of size crop_size that does not include any blurred block.
      4. Save the crop in output_dir if found.
    
    Parameters:
      - input_dir: Folder containing the original images.
      - output_dir: Folder where cropped images will be saved.
      - crop_size: Tuple (width, height) of the desired crop.
      - seed: Seed for the random number generator (for reproducibility).
      - blur_threshold: Variance threshold below which a block is considered blurred.
      - block_size: Size of the non-overlapping block used to detect blur.
      - max_attempts: Maximum number of random crop attempts per image.
    """
    # Set the random seed for reproducibility.
    random.seed(seed)
    os.makedirs(output_dir, exist_ok=True)
    
    # Collect image files from the input directory.
    image_extensions = ('*.jpg', '*.jpeg', '*.png', '*.bmp', '*.tiff')
    image_files = []
    for ext in image_extensions:
        image_files.extend(glob.glob(os.path.join(input_dir, ext)))
    image_files = sorted(image_files)
    
    for image_path in image_files:
        img = cv2.imread(image_path)
        if img is None:
            print(f"Could not load image {image_path}. Skipping.")
            continue
        
        height, width = img.shape[:2]
        crop_w, crop_h = crop_size
        
        # Skip images that are too small.
        if width < crop_w or height < crop_h:
            print(f"Skipping {os.path.basename(image_path)}: image too small ({width}x{height}).")
            continue
        
        # Convert the image to grayscale for blur detection.
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Detect blurred regions.
        blurred_regions = detect_blurred_regions(gray, block_size=block_size, blur_threshold=blur_threshold)
        
        valid_crop_found = False
        for attempt in range(max_attempts):
            # Choose a random crop position.
            crop_x = random.randint(0, width - crop_w)
            crop_y = random.randint(0, height - crop_h)
            crop_box = (crop_x, crop_y, crop_x + crop_w, crop_y + crop_h)
            
            valid = True  # Assume this crop is valid until proven otherwise.
            # Check if the candidate crop intersects any blurred block.
            for region in blurred_regions:
                if boxes_intersect(crop_box, region):
                    valid = False
                    break
            
            if valid:
                valid_crop_found = True
                break  # Accept this crop.
        
        if not valid_crop_found:
            print(f"No valid crop found for {os.path.basename(image_path)} after {max_attempts} attempts. Skipping.")
            continue
        
        # Crop and save the image.
        cropped_img = img[crop_y:crop_y+crop_h, crop_x:crop_x+crop_w]
        base_name = os.path.basename(image_path)
        output_path = os.path.join(output_dir, base_name)
        cv2.imwrite(output_path, cropped_img)
        print(f"Cropped {base_name} at ({crop_x}, {crop_y}) and saved to {output_path}")

if __name__ == "__main__":
    # Use raw strings for Windows paths to avoid escape sequence issues.
    input_folder = r"D:\Download\meta\Meta Images"    # Folder containing the original images.
    output_folder = r"D:\Download\meta\Cropped and Deblurred"     # Folder to save the cropped images.
    
    crop_images_avoiding_blur(input_folder, output_folder)


Cropped sa_1.jpg at (654, 228) and saved to D:\Download\meta\Cropped and Deblurred\sa_1.jpg
No valid crop found for sa_10.jpg after 100 attempts. Skipping.
No valid crop found for sa_100.jpg after 100 attempts. Skipping.
Cropped sa_1000.jpg at (252, 1185) and saved to D:\Download\meta\Cropped and Deblurred\sa_1000.jpg
No valid crop found for sa_10000.jpg after 100 attempts. Skipping.
No valid crop found for sa_10001.jpg after 100 attempts. Skipping.
Cropped sa_10002.jpg at (556, 343) and saved to D:\Download\meta\Cropped and Deblurred\sa_10002.jpg
No valid crop found for sa_10003.jpg after 100 attempts. Skipping.
No valid crop found for sa_10004.jpg after 100 attempts. Skipping.
No valid crop found for sa_10005.jpg after 100 attempts. Skipping.
Cropped sa_10006.jpg at (1187, 675) and saved to D:\Download\meta\Cropped and Deblurred\sa_10006.jpg
Cropped sa_10007.jpg at (1002, 886) and saved to D:\Download\meta\Cropped and Deblurred\sa_10007.jpg
No valid crop found for sa_10008.jpg after 